In [1]:
#Let's import all that will be needed
import os
import tarfile
from six.moves import urllib
import pandas as pd
import numpy as np

#Some methods useful for preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, cross_val_score, GridSearchCV #For stratified sampling

#The bases for our custom transformation
from sklearn.base import BaseEstimator, TransformerMixin

#Some models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#Visualization
%matplotlib inline
import matplotlib.pyplot as plt

#A metric for measuring RMSE
from sklearn.metrics import mean_squared_error

We fetch our data from githubusercontent

In [2]:
download_root = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
Housing_path = os.path.join("datasets", "housing")
Housing_url = download_root + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=Housing_url, housing_path=Housing_path):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
#Now the csv file is available for future usage
fetch_housing_data()

def load_housing_data(housing_path = Housing_path):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)
#We load our data
HousingData = load_housing_data()

We split the whole data set for training purposes and then stratified our data

In [3]:
train_set, test_set = train_test_split(HousingData, test_size=0.2, random_state=42)


HousingData["income_cat"] = np.ceil(HousingData["median_income"] / 1.5)
HousingData["income_cat"].where(HousingData["income_cat"] < 5, 5.0, inplace=True)

#We make stratification based on the income_cat that was obtained with median_income
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(HousingData, HousingData["income_cat"]):
    strat_train_set = HousingData.loc[train_index]
    strat_test_set = HousingData.loc[test_index]

Now we remove "income_cat" after stratifying the set. So we get our set as before. 

And create a copy of our housing data

In [4]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

housing = strat_train_set.copy()
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()


We separate our data into numerical and categorical and adapt the last one. 

Then we create a custom transform for adding extra attributes. and use the column transformer to transform categorical and numerical data correspondingly.

In [5]:
housing_num = housing.drop("ocean_proximity", axis = 1)

rooms_ix, bedrooms_ix, population_ix, households_ix = 4,5,6,7

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y = None):
        return self
    def transform(self,X,y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
#Transformations for numerical data        
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler()),
])

#We specify the attributes that are numerical and categorical
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

#Combining the previous transformation with the categorical one hot encoder
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

Let's start training and evaluating on the training set.

We use mean_squared_error to get this model's RMSE.

# Linear Regression Model:

In [6]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

#We gain an inside of prediction and real value
#some_data = housing.iloc[:10]
#some_labels = housing_labels.iloc[:10]
#some_data_prepared = full_pipeline.transform(some_data)
#for pred, real in zip(lin_reg.predict(some_data_prepared), list(some_labels)):
#    print("Prediction :", pred, "- Real Value :", real) 

housing_predictions = lin_reg.predict(housing_prepared)
lin_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))
lin_rmse

69036.32451019084

# Decision Tree Model:

In [7]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions_tree = tree_reg.predict(housing_prepared)
tree_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions_tree))
tree_rmse


0.0

Obviously there is some overfitting with this model.

# Random Forest Model:

In [8]:
forest_reg = RandomForestRegressor(n_estimators = 30, max_features = 8)
forest_reg.fit(housing_prepared, housing_labels)

housing_predictions_forest = forest_reg.predict(housing_prepared)
forest_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions_forest))
forest_rmse

19725.019242083607

# Cross-validation for avoiding overfitting
The previous result show us that there may be some data overfitting.

A great alternative is to use Scikit-Learn’s cross-validation feature. The following code performs K-fold cross-validation: it randomly splits the training set into 10 distinct subsets called folds, then it trains and evaluates the Decision Tree model 10 times, picking a different fold for evaluation every time and training on the other 9 folds.

In [9]:
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)

Scores: [ 72418.71148091  69270.30228297  72457.97254624  73909.73539962
  75577.09533313  76172.28473426  69741.82999752  67771.51688358
  74265.27134841  71573.5194082 ]
Mean: 72315.8239415
Standard deviation: 2626.44744073


We do the same for the linear regression model

In [10]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [ 67469.28784592  67422.28193468  68356.19928133  74785.20131013
  68241.17784274  71620.02931289  65379.21201448  68578.87113789
  73052.42414895  68092.44973869]
Mean: 69299.7134568
Standard deviation: 2753.0098942


We do the same for random forest

In [11]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [ 50014.9220298   47918.23575145  50339.47616208  52590.59272334
  50595.65081173  54773.21024976  49544.04872916  50155.54976259
  54175.9475945   51212.2893069 ]
Mean: 51131.9923121
Standard deviation: 2015.56247169


We see that random Forest is actually the most accurate model for predicting housing costs

# Support Vector Machines

In [12]:
from sklearn.svm import SVR

In [13]:
best_param = {'C': 157055.10989448498, 'gamma': 0.26497040005002437, 'kernel': 'rbf'} #I got these values from doing randomized search
VectorMach_reg = SVR(**best_param)
VectorMach_reg.fit(housing_prepared, housing_labels)

housing_predictions_VectorMach = VectorMach_reg.predict(housing_prepared)
VectorMach_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions_VectorMach))
VectorMach_rmse

50078.719873647897

So far SVM seems to be the best solution for predicting housing values.

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000024D41DADD30>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000024D45737278>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [15]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

50681.6021466 {'max_features': 7, 'n_estimators': 180}
53434.1465117 {'max_features': 5, 'n_estimators': 15}
52778.836354 {'max_features': 3, 'n_estimators': 72}
52537.4679252 {'max_features': 5, 'n_estimators': 21}
50762.126242 {'max_features': 7, 'n_estimators': 122}
52748.7755502 {'max_features': 3, 'n_estimators': 75}
52557.2964939 {'max_features': 3, 'n_estimators': 88}
51520.3047696 {'max_features': 5, 'n_estimators': 100}
52427.1765887 {'max_features': 3, 'n_estimators': 150}
67348.1147123 {'max_features': 5, 'n_estimators': 2}


We define the class for returning top k important features In case we want to build our model based on the top k important features.

In [16]:
def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature = feature_importances
        self.k = k
    def fit(self, X, y = None):
        self.feature_indices = indices_of_top_k(self.feature, self.k)
        return self
    def transform(self, X):
        return(X[:,self.feature_indices])

In [17]:
k = 5
Pipeline_important_features = Pipeline([
    ("full_pipeline", full_pipeline),
    ("feature_selection", FeatureSelector(feature_importances, k))
])

Housing_prepared_selectedfeatures = Pipeline_important_features.fit_transform(housing)

NameError: name 'feature_importances' is not defined

In [ ]:
Housing_prepared_selectedfeatures[0:3]

# Full_pipeline implementation

Creating a single pipeline that does the full data preparation plus the final prediction.

In [ ]:
Pipeline_important_features_prediction= Pipeline([
    ("full_pipeline", full_pipeline),
    ("feature_selection", FeatureSelector(feature_importances, k)),
    ("SVM_reg", SVR(**best_param))
])
Pipeline_important_features_prediction.fit(housing, housing_labels)

# Predictions vs labels comparison

In [ ]:
some_data = housing.iloc[:10]
some_labels = housing_labels.iloc[:10]

print("Predictions:\t", Pipeline_important_features_prediction.predict(some_data))
print("Labels:\t\t", list(some_labels))

We can see that our model is actually very accurate, compact version with only pipelines are to be done later on for this project.